<a href="https://colab.research.google.com/github/robertoleonie/TEA_2023.1/blob/main/C%C3%B3pia_de_2023_1_TEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Complexidade de Algoritmos - por que se importar?
- Eficiência (fazer mais com menos)
- Escalabilidade (há coisas que o dinheiro não pode comprar)
- Possibilidade de se comparar soluções antes mesmo de codá-las
- Ir bem em coding interviews (https://vigusmao.github.io/cursos/google_interview_tips.pdf)

## Notação O ("Big-O") na prática
Quantas operações serão executadas em cada trecho de código abaixo

In [ ]:
from time import time

def do_something_A(x):  # O(x^2)
    count = 0
    for i in range(1, x+1): 
        for j in range(1, x+1):  
            count += 1
    return count

def do_something_B(x):  # O(x^2)
    count = 0
    for i in range(1, x+1): 
        for j in range(i+1, x+1):  
            count += 1
    return count

def do_something_C(x):  # O(x log x)
    count = 0
    for i in range(1, x+1):  
        for j in range(1, x+1, i):
            count += 1
    return count

def do_something_D(x):  # O(x)
    count = 0
    while x > 0:
        count += 1
        x -= 1
    return count

def do_something_E(x):  # O(x)
    count = 0
    while x > 0:
        count += 1
        x -= 2
    return count

def do_something_F(x):  # O(log x)
    count = 0
    while x > 0:
        count += 1
        x //= 2
    return count

def do_something_G(x):  # O(2^x)
    def do_something_recursively(x):
        if x > 0:
            return 1 + do_something_recursively(x-1) + \
                       do_something_recursively(x-1)
        return 1
    return do_something_recursively(x)

Nota sobre `do_something_C`:

$\sum_{i=1,\ldots,x} x/i = x \sum_{i=1,\ldots,x} 1/i = x H(x) = O(x \log x)$,

onde $H(x)$ é o $x$-ésimo número harmônico, que é aproximadamente igual a $\ln x$.

Vamos checar.

Antes, vamos acrescentar alguma instrumentação.

In [ ]:
def run(functions, clock=False):
    while True:
        input_values = eval(input("Input values (iterable): "))
        if input_values == 0:
            break
        
        # processes each input value
        results = []
        for n in input_values:
            row = [str(n)]
            for f in functions:
                start = time()
                result = str(f[0](n))
                duration = time() - start
                if clock:
                    result += ' (' + ("%.6f" % duration) + ')'
                row.append(result)
            results.append(row)

        # computes the maximum length on each column
        lengths = [max((len("%s" % row[column]) for row in results)) 
            for column in range(len(functions) + 1)]    

        # prints the results
        msg_format = "".join(
            ["%%%d" % max(11, length + 2) + "s" for length in lengths])
        print("\n" + msg_format % tuple(["n"] + [f[1] for f in functions]))  
        for row in results:
            print(msg_format % tuple(row))
        print("\n")


E agora vamos rodar.

In [ ]:
functions = [
    (do_something_A, "O(x^2)"),
    (do_something_B, "O(x^2)"),
    (do_something_C, "O(x log x)"),
    (do_something_D, "O(x)"),
    (do_something_E, "O(x)"),
    (do_something_F, "O(log x)"),
    # (do_something_G, "O(2^x)"),
    ]

run(functions, clock=False)

P: Mas e o verdadeiro custo de cada "operação"?

R: Esqueça. Não vai fazer diferença.

P: E as constantes multiplicativas e aditivas?

R: Não damos a mínima.

**Obs.: Quando estiver comparando algoritmos com complexidades assintóticas distintas, e se você tem preocupações sobre escalabilidade, isto é, se suas entradas não são ridiculamente pequenas.**

Vamos ver agora os valores produzidos por algumas funções comuns usadas para denotar complexidades assintóticas. Podemos brincar com elas um pouco, adicionando constantes.


In [ ]:
from math import *

def exponential(n):
    return 2**n

def cubic(n):
    return n**3
    
def quadratic(n):
    return 3 * n**2 + 10*n
    
def nlogn(n):
    return 100*n * ceil(log(n, 2)) + 5000
    
def linear(n):
    return 10*n+200
    
def sqrtn(n):
    return 80*ceil(sqrt(n))
    
def logn(n):
    return 100*ceil(log(n, 2))
    
def constant(n):
    return 1500

functions = [
    # (exponential, "O(2^n)"),
    (cubic, "O(n^3)"),
    (quadratic, "O(n^2)"),
    (nlogn, "O(n log n)"),
    (linear, "O(n)"),
    (sqrtn, "O(sqrt n)"),
    (logn, "O(log n)"),
    (constant, "O(1)"),
    ]

run(functions, clock=False)

Input values (iterable): (10**k for k in range(10))

           n                        O(n^3)               O(n^2)     O(n log n)         O(n)  O(sqrt n)   O(log n)       O(1)
           1                             1                 1003           5000          210         80          0         15
          10                          1000                 1300           9000          300        320        400         15
         100                       1000000                31000          75000         1200        800        700         15
        1000                    1000000000              3001000        1005000        10200       2560       1000         15
       10000                 1000000000000            300001000       14005000       100200       8000       1400         15
      100000              1000000000000000          30000001000      170005000      1000200      25360       1700         15
     1000000           1000000000000000000        3000000001000     2000

KeyboardInterrupt: ignored

## Exemplo 1: O problema dos armários

Em um vestiário, existem $n$ armários inicialmente fechados e numerados de 1 a $n$. Existem também $n$ crianças. A criança número 1 entra no vestiário e abre todos os armários. A seguir, a criança 2 entra no vestiário e fecha todos os armários que são múltiplos de 2. Entra então a criança 3 e muda o estado de todos os armários que são múltiplos de 3. E assim sucessivamente, com a criança $j$ mudando o estado de todos os armários que são múltiplos de $j$, para todo $j$ de 1 a $n$. No final, após a $n$-ésima criança terminar, que armários estarão abertos? Como você abordaria esse problema computacionalmente?

In [ ]:
from math import sqrt

def lockers_A(n):  # O(n^2)
    states = [False] * (n+1)
    for kid in range(1, n+1):
        for locker in range(1, n+1):
            if locker % kid == 0:
                states[locker] = not states[locker]
    count = 0
    for s in states:
        if s:
            count += 1
    return count

def lockers_B(n):  # O(n log n)
    states = [False] * (n+1)
    for kid in range(1, n+1):
        for locker in range(kid, n+1, kid):
            states[locker] = not states[locker]
    count = 0
    for s in states:
        if s:
            count += 1
    return count

def lockers_C(n):  # O(n)
    count = 0
    for locker in range(1, n+1):
        sqrroot = int(sqrt(locker))
        if sqrroot * sqrroot == locker:
            count += 1
    return count

def lockers_D(n):  # O(n^0.5)
    count = 0
    base = 1
    locker = 1
    while locker <= n:
        count += 1
        base += 1
        locker = base * base
    return count

def lockers_E(n):
    return int(sqrt(n))


functions = [
    # (lockers_A, "O(n^2)"),
    # (lockers_B, "O(n log n)"),
    (lockers_C, "O(n)"),
    (lockers_D, "O(sqrt(n))"),
    (lockers_E, "O(1)"),
    ]

run(functions, True)

Input values (iterable): 1000000, 2000000, 4000000

          n             O(n)       O(sqrt(n))             O(1)
    1000000  1000 (0.233060)  1000 (0.000167)  1000 (0.000002)
    2000000  1414 (0.423854)  1414 (0.000230)  1414 (0.000002)
    4000000  2000 (1.818996)  2000 (0.000623)  2000 (0.000003)




KeyboardInterrupt: ignored

## Exemplo 2: O problema da celebridade
Em uma festa, defina uma celebridade como sendo alguém que é conhecido por todos os convidados, mas que não conhece nenhum deles. Dada uma matriz booleana $N$x$N$, onde a célula $i,j$ indica se o convidado $i$ conhece o convidado $j$, como você determinaria se existe na festa alguma celebridade?




In [ ]:
from random import getrandbits
from time import time

def build_zeroes_matrix(n):
    return [[False]*n for row in range(n)]

def build_random_matrix(n):
    matrix = []
    for row_idx in range(n):
        row = []
        for col_idx in range(n):
            row.append(getrandbits(1) == 1)
        matrix.append(row)
    return matrix

def force_celebrity(matrix, celeb_idx):
    n = len(matrix)
    for guest in range(n):
        matrix[celeb_idx][guest] = False
        matrix[guest][celeb_idx] = True

def check_celebrity(matrix, candidate):
    n = len(matrix)

    # verify row
    for guest in range(n):
        if guest == candidate:
            continue
        if matrix[candidate][guest]:
            return False

    # verify column
    for guest in range(n):
        if guest == candidate:
            continue
        if not matrix[guest][candidate]:
            return False  
              
    return True

def find_celebrity_A(matrix):  # O(n^2)
    n = len(matrix)
    for candidate in range(n):
        if check_celebrity(matrix, candidate):
            return candidate
    return None

def find_celebrity_B(matrix):  # O(n)
    n = len(matrix)
    candidate = 0
    for guest in range(1, n):
        if matrix[candidate][guest]:
            candidate = guest
    if check_celebrity(matrix, candidate):
        return candidate
    return None

for n in (1000 * 2**k for k in range(4)):
  m = build_zeroes_matrix(n)
  force_celebrity(m, n-1)

  print("\n\nn =", n)
  start = time()
  print(find_celebrity_A(m))
  print(time() - start)
  start = time()
  print(find_celebrity_B(m))
  print(time() - start)






n = 1000
999
0.3041250705718994
999
0.0007603168487548828


n = 2000
1999
0.3626387119293213
1999
0.0009808540344238281


n = 4000
3999
1.4782803058624268
3999
0.0021131038665771484


n = 8000
7999
5.79651951789856
7999
0.004133701324462891
